In [1]:
#%matplotlib inline
!pip install openml

In [2]:
import sys

if sys.platform == "win32":  # noqa
    print(
        "The pyrfr library (requirement of fanova) can currently not be installed on Windows systems"
    )
    exit()

import json
#import fanova
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import openml
import uuid
import numpy as np
import openml
from tabulate import tabulate
import numpy as np
import openml
from openml.extensions.sklearn import cat, cont
import sklearn
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import VarianceThreshold

from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from pprint import pprint

from pandas import read_csv                           # For dataframes
from pandas import DataFrame                       # For dataframes
from numpy import ravel                                  # For matrices
import matplotlib.pyplot as plt                        # For plotting data
import seaborn as sns                                     # For plotting data
from sklearn.model_selection import train_test_split    # For train/test splits
from sklearn.neighbors import KNeighborsClassifier    # The k-nearest neighbor classifier
from sklearn.feature_selection import VarianceThreshold # Feature selector
from sklearn.pipeline import Pipeline                                  # For setting up pipeline
# Various pre-processing steps
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV      # For optimization


**Get openml studies **

In [3]:
suites = openml.study.list_suites(output_format="dataframe", status="all")
print(tabulate(suites.tail(100), headers='keys', tablefmt='psql'))
suite = openml.study.get_suite(218) #choose automl study
print(suite)
print(suite.description)
print(suite.tasks)

+-----+------+----------------------------------+--------------------+-------------------------------------------------------+----------------+---------------------+-----------+
|     |   id | alias                            | main_entity_type   | name                                                  | status         | creation_date       |   creator |
|-----+------+----------------------------------+--------------------+-------------------------------------------------------+----------------+---------------------+-----------|
|  14 |   14 | OpenML100                        | task               | Collaborative, reproducible benchmarking and analysis | in_preparation | 2019-02-21 18:40:13 |         1 |
|  99 |   99 | OpenML-CC18                      | task               | OpenML-CC18 Curated Classification benchmark          | active         | 2019-02-21 18:47:13 |         1 |
| 218 |  218 | AutoML-Benchmark                 | task               | AutoML Benchmark                       

filter Auto ml data set

In [4]:
Data_df = openml.datasets.list_datasets(output_format="dataframe",data_id=suite.data)
Data_df= Data_df.query("NumberOfInstancesWithMissingValues == 0")
Data_df=Data_df.query("NumberOfInstances > 5000")
print(tabulate(Data_df.tail(200), headers='keys', tablefmt='psql'))
Dataid_array = Data_df.did.to_numpy()
print(Dataid_array)

+-------+-------+----------------------------------------+-----------+------------+----------+----------+---------------------+-------------------------------+---------------------+-------------------+--------------------+---------------------+--------------------------------------+-------------------------+---------------------------+----------------------------+
|       |   did | name                                   |   version |   uploader | status   | format   |   MajorityClassSize |   MaxNominalAttDistinctValues |   MinorityClassSize |   NumberOfClasses |   NumberOfFeatures |   NumberOfInstances |   NumberOfInstancesWithMissingValues |   NumberOfMissingValues |   NumberOfNumericFeatures |   NumberOfSymbolicFeatures |
|-------+-------+----------------------------------------+-----------+------------+----------+----------+---------------------+-------------------------------+---------------------+-------------------+--------------------+---------------------+----------------------

In [5]:
#take only the tasks that contain the data sets after filtering them to - without missing value& number of rows
#get task id for filter data sets
tasks = openml.tasks.list_tasks(
    status="active",
    output_format="dataframe",
)

task_ids = []
for did in Dataid_array:
  tasks_ = list(tasks.query("did == {}".format(did)).tid)
  task_ids.append(tasks_)
print(task_ids) #task we will check

[[7275, 75127, 127070, 147936, 168886, 169958, 189354, 191465, 213139, 233136, 234254, 256377, 256388, 297527, 318654, 339809], [14965, 56795, 56796, 56797, 56798, 56799, 56800, 56801, 56802, 125908, 126029, 127242, 145833, 146585, 146755, 148108, 168876, 170130, 191637, 213311, 233110, 234426, 256721, 256732, 297699, 318826, 339981, 359982], [9977, 56383, 56384, 56385, 56386, 56387, 56388, 56389, 56390, 125896, 126026, 127265, 145854, 146595, 146768, 148131, 168805, 170153, 191660, 213334, 233107, 234449, 256767, 256778, 297722, 318849, 340004, 359980], [9952, 54999, 55000, 55001, 55002, 55003, 55004, 55005, 55006, 75227, 125884, 127268, 145857, 146770, 148134, 167190, 168350, 168840, 170156, 191663, 213337, 233103, 234452, 256773, 256784, 297725, 318852, 340007], [7593, 75193, 127371, 146155, 146453, 148237, 167423, 167781, 168139, 168552, 170259, 189560, 190179, 191766, 211453, 213440, 233100, 234555, 256979, 256990, 297828, 318955, 340110, 360338], [9911, 10044, 34539, 56943, 56944

Get data set by task id

In [14]:
#get data from task id
#amazon
task_id = 34539
task = openml.tasks.get_task(task_id)
X, y = task.get_X_and_y(dataset_format="dataframe")
n_repeats, n_folds, n_samples = task.get_split_dimensions()
print(
    "Task {}: number of repeats: {}, number of folds: {}, number of samples {}.".format(
        task_id, n_repeats, n_folds, n_samples,
    )
)

Task 34539: number of repeats: 1, number of folds: 10, number of samples 1.


In [15]:
# take all the evaluations (score and parameters for the data sets)
evals_df=pd.DataFrame(columns=['run_id', 'task_id', 'setup_id', 'flow_id', 'flow_name', 'data_id',
       'data_name', 'function', 'upload_time', 'uploader', 'uploader_name',
       'value', 'values', 'array_data', 'parameters'])

suite = openml.study.get_suite(218) #AUTOML STUDY

evaluation_measure = "predictive_accuracy"
limit_per_task = 500
limit_nr_tasks = 15
n_trees = 16

fanova_results = []
# we will obtain all results from OpenML per task. Practice has shown that this places the bottleneck on the
# communication with OpenML, and for iterated experimenting it is better to cache the results in a local file.
for did in Dataid_array:
  tasks_ = list(tasks.query("did == {}".format(did)).tid)
  #for idx, task_id in enumerate(suite.tasks):
  for idx, task_id in enumerate(tasks_):
      if limit_nr_tasks is not None and idx >= limit_nr_tasks:
          continue
      print(
          "Starting with task %d (%d/%d)"
          % (task_id, idx + 1, len(suite.tasks) if limit_nr_tasks is None else limit_nr_tasks)
      )
      # note that we explicitly only include tasks from the benchmark suite that was specified (as per the for-loop)
      evals = openml.evaluations.list_evaluations_setups(
          evaluation_measure,
          #flows=[flow_id],
          tasks=[task_id],
          size=limit_per_task,
          output_format="dataframe",
          #parameters_in_separate_columns=True
      )

      performance_column = "value"
      evals_df = evals_df.append(evals,ignore_index=True)


Starting with task 7275 (1/15)
Starting with task 75127 (2/15)
Starting with task 127070 (3/15)
Starting with task 147936 (4/15)
Starting with task 168886 (5/15)
Starting with task 169958 (6/15)
Starting with task 189354 (7/15)
Starting with task 191465 (8/15)
Starting with task 213139 (9/15)
Starting with task 233136 (10/15)
Starting with task 234254 (11/15)
Starting with task 256377 (12/15)
Starting with task 256388 (13/15)
Starting with task 297527 (14/15)
Starting with task 318654 (15/15)
Starting with task 14965 (1/15)
Starting with task 56795 (2/15)
Starting with task 56796 (3/15)
Starting with task 56797 (4/15)
Starting with task 56798 (5/15)
Starting with task 56799 (6/15)
Starting with task 56800 (7/15)
Starting with task 56801 (8/15)
Starting with task 56802 (9/15)
Starting with task 125908 (10/15)
Starting with task 126029 (11/15)
Starting with task 127242 (12/15)
Starting with task 145833 (13/15)
Starting with task 146585 (14/15)
Starting with task 146755 (15/15)
Starting w

Filter for random forest classifier tasks

In [16]:
#filter fot task runing on random forest
filter_random_forest_df=  evals_df.loc[evals_df['flow_name'].str.contains("RandomForest", case=False)]
print(tabulate(filter_random_forest_df.tail(200), headers='keys', tablefmt='psql'))


+------+----------+-----------+------------+-----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+----------------------------------------+---------------------+---------------------+------------+-----------------+----------+----------+--------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Get top score for each data set that run with lernrt=random forest

In [17]:
#get for each data set top 1 score
df_agg = filter_random_forest_df.groupby(['data_name','data_id','task_id','flow_id','setup_id']).agg({'value':max})
g = (df_agg['value'].groupby('data_id', group_keys=False))

res = g.apply(lambda x: x.sort_values(ascending=False).head(1))


# enter top 1 score into data frame
final_evals_df=pd.DataFrame()

for did in Dataid_array:
  chek_sort=filter_random_forest_df.query("data_id=="+str(did)).sort_values(by=['value'],ascending=False).head(1)
  final_evals_df = final_evals_df.append(chek_sort,ignore_index=True)

In [18]:
res.head(100)

data_name                               data_id  task_id  flow_id  setup_id
bank-marketing                          1461     14965    6969     4000705     0.906837
nomao                                   1486     9977     4030     6411        0.969882
phoneme                                 1489     9952     1726     2369        0.915248
covertype                               1596     75193    9561     8086271     0.939624
Amazon_employee_access                  4135     34539    6969     4029256     0.944155
numerai28.6                             23517    167120   8315     6877836     0.519705
connect-4                               40668    146195   7096     4703788     0.827997
shuttle                                 40685    146212   16118    8219391     0.999897
Fashion-MNIST                           40996    146825   8351     7002643     0.889114
jungle_chess_2pcs_raw_endgame_complete  41027    167119   8315     6882479     0.831924
MiniBooNE                               4115

Get task num iteration for test train loop

In [19]:
#get data from task id
#amazon
task_id = 34539
task = openml.tasks.get_task(task_id)
X, y = task.get_X_and_y(dataset_format="dataframe")
n_repeats, n_folds, n_samples = task.get_split_dimensions()
print(
    "Task {}: number of repeats: {}, number of folds: {}, number of samples {}.".format(
        task_id, n_repeats, n_folds, n_samples,
    )
)

Task 34539: number of repeats: 1, number of folds: 10, number of samples 1.


get hyperpara in diffrent column for the run that in top 1 for spesific data set

In [20]:
# Return evaluations for flow_id in descending order based on predictive_accuracy
# with hyperparameters. parameters_in_separate_columns returns parameters in
# separate columns
flows = final_evals_df.flow_id.to_numpy()
setups = final_evals_df.setup_id.to_numpy()
tasks = final_evals_df.task_id.to_numpy()
evaluation_measure = "predictive_accuracy"

#change for each data set -flows,setups,tasks by res data set
# amazon data set- random forest best hyperparam
evals_setups = openml.evaluations.list_evaluations_setups(
    function=evaluation_measure,
    flows=[6969],     
    setups=[4029256],
    tasks=[34539],
    parameters_in_separate_columns=True
)

In [21]:

print(tabulate(evals_setups.tail(200), headers='keys', tablefmt='psql'))

+----+----------+-----------+------------+-----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+------------------------+---------------------+---------------------+------------+-----------------+----------+----------+--------------+----------------------------------------------------------------+-------------------------------------------------------------------+----------------------------------------------------------------+----------------------------------------------------------------+-------------------------------------------------------------------+---------------------------------------------------------------------+-------------------------------------------------------------------------+----------------------------------------

In [22]:
       
evaluation_measure = "predictive_accuracy"
# janis- random forest best hyperparam
# task_id  flow_id  setup_id
# 168330   16119    8219401
evals_setups = openml.evaluations.list_evaluations_setups(
    function=evaluation_measure,
    flows=[16119 ],     
    setups=[8219401],
    tasks=[168330],
    parameters_in_separate_columns=True
)
print(tabulate(evals_setups.tail(200), headers='keys', tablefmt='psql'))

+----+----------+-----------+------------+-----------+--------------------------------------+-----------+-------------+---------------------+---------------------+------------+-----------------+----------+----------+--------------+---------------------------------------------+-----------------------------------------------+----------------------------------------------+
|    |   run_id |   task_id |   setup_id |   flow_id | flow_name                            |   data_id | data_name   | function            | upload_time         |   uploader | uploader_name   |    value | values   | array_data   |   automlbenchmark_tunedrandomforest(1)_time |   automlbenchmark_tunedrandomforest(1)_memory |   automlbenchmark_tunedrandomforest(1)_cores |
|----+----------+-----------+------------+-----------+--------------------------------------+-----------+-------------+---------------------+---------------------+------------+-----------------+----------+----------+--------------+-----------------------

In [23]:
evaluation_measure = "predictive_accuracy"
# helena- random forest best hyperparam
# task_id  flow_id  setup_id
# 168329   16118    8219391
evals_setups = openml.evaluations.list_evaluations_setups(
    function=evaluation_measure,
    flows=[16118 ],     
    setups=[8219391],
    tasks=[168329],
    parameters_in_separate_columns=True
)
print(tabulate(evals_setups.tail(200), headers='keys', tablefmt='psql'))

+----+----------+-----------+------------+-----------+---------------------------------+-----------+-------------+---------------------+---------------------+------------+-----------------+----------+----------+--------------+----------------------------------------+------------------------------------------+-----------------------------------------+
|    |   run_id |   task_id |   setup_id |   flow_id | flow_name                       |   data_id | data_name   | function            | upload_time         |   uploader | uploader_name   |    value | values   | array_data   |   automlbenchmark_randomforest(1)_time |   automlbenchmark_randomforest(1)_memory |   automlbenchmark_randomforest(1)_cores |
|----+----------+-----------+------------+-----------+---------------------------------+-----------+-------------+---------------------+---------------------+------------+-----------------+----------+----------+--------------+----------------------------------------+---------------------------

In [24]:
evaluation_measure = "predictive_accuracy"
# Fasion Mnist- random forest best hyperparam
# task_id  flow_id  setup_id
# 146825   8351     7002643
evals_setups = openml.evaluations.list_evaluations_setups(
    function=evaluation_measure,
    flows=[8351 ],     
    setups=[7002643],
    tasks=[146825],
    parameters_in_separate_columns=True
)
print(tabulate(evals_setups.tail(200), headers='keys', tablefmt='psql'))

+----+----------+-----------+------------+-----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+---------------+---------------------+---------------------+------------+-----------------+----------+----------+--------------+---------------------------------------------------------------------+-----------------------------------------------------------------+---------------------------------------------------------------+---------------------------------------------------------+-------------------------------------------------------+--------------------------------------------------------------------------------+----------------------------------------------------------------+-------------------------------------------------------------------+-

recreate this task pipline- same parameter used in the open ml  

In [25]:
#Fasion mnist best param random forest classifier
cat_imp = make_pipeline(OneHotEncoder(handle_unknown="ignore", sparse=False))
cont_imp = SimpleImputer(strategy="most_frequent")
ct = ColumnTransformer([("cat", cat_imp, cat), ("cont", cont_imp, cont)])
param_dict = {'n_estimators': 300,
               'max_features':  0.0345443,
               'min_samples_split': 2,
               'min_samples_leaf': 3,
               'criterion':'gini',
               'bootstrap': 'false',
               'random_state':96495  
               }
pipline=make_pipeline(
    ColumnTransformer([("cat", cat_imp, cat), ("cont", cont_imp, cont)])
    ,VarianceThreshold()
    ,RandomForestClassifier(**param_dict)
    
    )
pprint(pipline)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <function cat at 0x11ff8edc0>),
                                                 ('cont',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  <function cont at 0x11ff8e1f0>)])),
                ('variancethreshold', VarianceThreshold()),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap='false',
                                        max_features=0.0345443,
                                        min_samples_leaf=3, n_estimators=

In [26]:
#Amazom best param random forest
cat_imp = make_pipeline(OneHotEncoder(handle_unknown="ignore", sparse=False))
cont_imp = SimpleImputer(strategy="most_frequent")
ct = ColumnTransformer([("cat", cat_imp, cat), ("cont", cont_imp, cont)])
param_dict = {'n_estimators': 100,
               'max_features':  0.116019,
               'min_samples_split': 15,
               'min_samples_leaf': 13,
               'criterion':'entropy',
               'bootstrap': 'false',
               'random_state':65266 
               }
pipline=make_pipeline(
    ColumnTransformer([("cat", cat_imp, cat), ("cont", cont_imp, cont)])
    ,VarianceThreshold()
    ,RandomForestClassifier(**param_dict)
    
    )

Finel loop foe score with task we want to check- in this case amazone-random forest-top 1 score params

In [ ]:
for repeat_idx in range(n_repeats):
    for fold_idx in range(n_folds):
        for sample_idx in range(n_samples):
            train_indices, test_indices = task.get_train_test_split_indices(
                repeat=repeat_idx, fold=fold_idx, sample=sample_idx,
            )
            X, y = task.get_X_and_y(dataset_format="dataframe")
            X_train = X.iloc[train_indices]
            y_train = y.iloc[train_indices]
            X_test = X.iloc[test_indices]
            y_test = y.iloc[test_indices]
            cat_imp = make_pipeline(OneHotEncoder(handle_unknown="ignore", sparse=False))
            # create pipline
            cont_imp = SimpleImputer(strategy="most_frequent")
            ct = ColumnTransformer([("cat", cat_imp, cat), ("cont", cont_imp, cont)])
            param_dict = {'n_estimators': 100,
                          'max_features':  0.116019,
                          'min_samples_split': 15,
                          'min_samples_leaf': 13,
                          'criterion':'entropy',
                          'bootstrap': 'false',
                          'random_state':65266 
                          }

            pipline=make_pipeline(
                ColumnTransformer([("cat", cat_imp, cat), ("cont", cont_imp, cont)])
                ,VarianceThreshold()
                ,RandomForestClassifier(**param_dict)
                
                )

            #second
            pipline.fit(X_train, y_train)
            score_l=pipline.score(X_test, y_test)
            #print('score: {0:.2f}'.format(score_l))
            print('score:',score_l )

            print(
                "Repeat #{}, fold #{}, samples {}: X_train.shape: {}, "
                "y_train.shape {}, X_test.shape {}, y_test.shape {}".format(
                    repeat_idx,
                    fold_idx,
                    sample_idx,
                    X_train.shape,
                    y_train.shape,
                    X_test.shape,
                    y_test.shape,
                )
            )

score: 0.9420201403722918
Repeat #0, fold #0, samples 0: X_train.shape: (29492, 9), y_train.shape (29492,), X_test.shape (3277, 9), y_test.shape (3277,)


Try random serch to get this param - random forest hyperparam min&max value like in the Article

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')
# import sys
# sys.path.append('/content/drive/My Drive/MorTeza-main/TEZA/code')

path='/Users/zlililaor/Desktop/MorTeza-main/TEZA/Trials/OpenMl_Grid_Trials '

In [ ]:
#AMAZON
# Number of trees in random forest
task_id = 34539
task = openml.tasks.get_task(task_id)
n_estimators = [int(x) for x in np.linspace(start = 2, stop = 100, num = 10)]
# Number of features to consider at every split
max_features =[0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1] #[(x) for x in np.linspace(start = 0.0, stop = 1, num = 10)]

# Minimum number of samples required to split a node
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 20, num = 12)]
# Minimum number of samples required at each leaf node
min_samples_leaf = [int(x) for x in np.linspace(start = 1, stop = 20, num = 13)]
# Method of selecting samples for training each tree
bootstrap = [True, False]
criterion=['entropy', 'gini']

# Create the random grid
# randomforestclassifier__ --> pprint(pipline.steps) #step names + __ need to be added to the parameter name
grid_param = { 'randomforestclassifier__n_estimators': n_estimators,
               'randomforestclassifier__max_features': max_features,
               'randomforestclassifier__min_samples_split': min_samples_split,
               'randomforestclassifier__min_samples_leaf': min_samples_leaf,
               'randomforestclassifier__criterion':criterion,
               'randomforestclassifier__bootstrap': [True, False]    
               }
                #random state of amazon random forest best classifier

#create pipline               
cat_imp = make_pipeline(OneHotEncoder(handle_unknown="ignore", sparse=False))
# create pipline
cont_imp = SimpleImputer(strategy="most_frequent")
ct = ColumnTransformer([("cat", cat_imp, cat), ("cont", cont_imp, cont)])

pipline=make_pipeline(
ColumnTransformer([("cat", cat_imp, cat), ("cont", cont_imp, cont)]),
VarianceThreshold(),
RandomForestClassifier()

)

#splite to trin test
X, y = task.get_X_and_y(dataset_format="dataframe")
train_indices, test_indices = task.get_train_test_split_indices()


X_train = X.iloc[train_indices]
y_train = y.iloc[train_indices]
X_test = X.iloc[test_indices]
y_test = y.iloc[test_indices]


grid = GridSearchCV(pipline,grid_param, cv=2,verbose=100)
grid.fit(X_train, y_train)
print('Training set score: ' + str(grid.score(X_train, y_train)))
print('Test set score: ' + str(grid.score(X_test, y_test)))
 
# Access the best set of parameters
best_params = grid.best_params_
print(best_params)
# Stores the optimum model in best_pipe
best_pipe = grid.best_estimator_
print(best_pipe)


result_df = DataFrame.from_dict(grid.cv_results_, orient='columns') #grid_scores_
result_df.to_csv(chkpt_dir)
print(result_df.columns)
print(tabulate(result_df.tail(200), headers='keys', tablefmt='psql'))



In [5]:
path='/Users/zlililaor/Desktop/MorTeza-main/TEZA/Trials/OpenMl_Grid_Trials '
#Fasion mnist
task_id = 146825
chkpt_dir='/content/drive/My Drive/MorTeza-main/TEZA/config changing tests/Grid_trials_fasionMnist.csv'
task = openml.tasks.get_task(task_id)

n_estimators = [int(x) for x in np.linspace(start = 2, stop = 500, num = 50)]
# Number of features to consider at every split
#max_features =[0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1] #[(x) for x in np.linspace(start = 0.0, stop = 1, num = 10)]

# Minimum number of samples required to split a node
#min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 20, num = 12)]
# Minimum number of samples required at each leaf node
#min_samples_leaf = [int(x) for x in np.linspace(start = 1, stop = 20, num = 13)]
# Method of selecting samples for training each tree
bootstrap = [False]
criterion=['gini']

# Create the random grid
# randomforestclassifier__ --> pprint(pipline.steps) #step names + __ need to be added to the parameter name
grid_param = { 'randomforestclassifier__n_estimators': n_estimators,
               'randomforestclassifier__criterion':criterion,
               'randomforestclassifier__bootstrap': bootstrap  
               }
                #random state of amazon random forest best classifier

#create pipline               
cat_imp = make_pipeline(OneHotEncoder(handle_unknown="ignore", sparse=False))
# create pipline
cont_imp = SimpleImputer(strategy="most_frequent")
ct = ColumnTransformer([("cat", cat_imp, cat), ("cont", cont_imp, cont)])

pipline=make_pipeline(
ColumnTransformer([("cat", cat_imp, cat), ("cont", cont_imp, cont)]),
VarianceThreshold(),
RandomForestClassifier()

)

#splite to trin test
X, y = task.get_X_and_y(dataset_format="dataframe")
train_indices, test_indices = task.get_train_test_split_indices()


X_train = X.iloc[train_indices]
y_train = y.iloc[train_indices]
X_test = X.iloc[test_indices]
y_test = y.iloc[test_indices]


grid = GridSearchCV(pipline,grid_param,verbose=1,cv=2) #, cv=5
grid.fit(X_train, y_train)
print('Training set score: ' + str(grid.score(X_train, y_train)))
print('Test set score: ' + str(grid.score(X_test, y_test)))
 
# Access the best set of parameters
best_params = grid.best_params_
print(best_params)
# Stores the optimum model in best_pipe
best_pipe = grid.best_estimator_
print(best_pipe)


result_df = DataFrame.from_dict(grid.cv_results_, orient='columns') #grid_scores_
result_df.to_csv(path)
print(result_df.columns)
print(tabulate(result_df.tail(200), headers='keys', tablefmt='psql'))

Fitting 2 folds for each of 50 candidates, totalling 100 fits


KeyboardInterrupt: 

In [ ]:
a=RandomForestClassifier()
pprint(a.get_params())

In [ ]:
!pip install matplotlib

In [ ]:
#Fasion mnist
#task_id = 146825
#robert
print('start')
task_id=168332
chkpt_dir='/Users/zlililaor/Desktop/MorTeza-main/TEZA/Trials/Grid_trials_robert_all_params '
#chkpt_dir='/content/drive/My Drive/MorTeza-main/TEZA/config changing tests/Grid_trials_robert_all_params.csv'
task = openml.tasks.get_task(task_id)

n_estimators = [int(x) for x in np.linspace(start = 2, stop = 100, num = 70)] #100
# Number of features to consider at every split
max_features =[0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1] #[(x) for x in np.linspace(start = 0.0, stop = 1, num = 10)]

# Minimum number of samples required to split a node
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 20, num = 12)]
# Minimum number of samples required at each leaf node
min_samples_leaf = [int(x) for x in np.linspace(start = 1, stop = 20, num = 13)]
# Method of selecting samples for training each tree
bootstrap = [False,True]
criterion=['gini','entropy']

# Create the random grid
# randomforestclassifier__ --> pprint(pipline.steps) #step names + __ need to be added to the parameter name
grid_param = { 'n_estimators': n_estimators,
              'n_jobs':[-1],
              'max_features':max_features,
              'min_samples_split':min_samples_split,
              'min_samples_leaf':min_samples_leaf,
              'bootstrap':bootstrap,
              'criterion':criterion,
              'random_state':[42]
               }
                #random state of amazon random forest best classifier

rf= RandomForestClassifier()
#splite to trin test
X, y = task.get_X_and_y(dataset_format="dataframe")
train_indices, test_indices = task.get_train_test_split_indices()


X_train = X.iloc[train_indices]
y_train = y.iloc[train_indices]
X_test = X.iloc[test_indices]
y_test = y.iloc[test_indices]
print('starting!')

grid = GridSearchCV(rf,grid_param,cv=3,verbose=1) #, cv=5 ,verbose=100
grid.fit(X_train, y_train)
print('Training set score: ' + str(grid.score(X_train, y_train)))
print('Test set score: ' + str(grid.score(X_test, y_test)))

# Access the best set of parameters
best_params = grid.best_params_
print(best_params)
# Stores the optimum model in best_pipe
best_pipe = grid.best_estimator_
print(best_pipe)


result_df = DataFrame.from_dict(grid.cv_results_, orient='columns') #grid_scores_
result_df.to_csv(chkpt_dir)
print(result_df.columns)
print(tabulate(result_df.tail(200), headers='keys', tablefmt='psql'))

start
starting!
Fitting 3 folds for each of 436800 candidates, totalling 1310400 fits
